# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

df=pd.read_csv("fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df1=df[(df['Date']<'2015-01-01')]
df2=df[(df['Date']>='2015-01-01')]

max=df1.loc[df1.groupby(['Day','Month'])['Data_Value'].idxmax(),:].reset_index()
min=df1.loc[df1.groupby(['Day','Month'])['Data_Value'].idxmin(),:].reset_index()
final=pd.merge(max,min,on=['Day','Month'])
final.rename(columns={'Data_Value_x':'TMAX','Data_Value_y':'TMIN','Date_x':'Date'},inplace=True)
final.drop(columns=['ID_x','ID_y','Element_x','Element_y','index_x','index_y','Date_y','Month','Day'],inplace=True)
final['TMAX']*=0.1
final['TMIN']*=0.1
final['Date']=final['Date'].apply(lambda x: x[5:])
final.sort_values(by='Date',inplace=True)

max2=df2.loc[df2.groupby(['Day','Month'])['Data_Value'].idxmax(),:].reset_index()
min2=df2.loc[df2.groupby(['Day','Month'])['Data_Value'].idxmin(),:].reset_index()
final2=pd.merge(max2,min2,on=['Day','Month'])
final2.rename(columns={'Data_Value_x':'TMAX','Data_Value_y':'TMIN','Date_x':'Date'},inplace=True)
final2.drop(columns=['ID_x','ID_y','Element_x','Element_y','index_x','index_y','Date_y','Month','Day'],inplace=True)
final2['TMAX']*=0.1
final2['TMIN']*=0.1
final2['Date']=final2['Date'].apply(lambda x: x[5:])
final2.sort_values(by='Date',inplace=True)

compare=pd.merge(final, final2, on=['Date'])
compare_high=compare[(compare['TMAX_y']>compare['TMAX_x'])]
compare_low=compare[(compare['TMIN_y']<compare['TMIN_x'])]

plt.figure()
plt.plot(final['Date'],final['TMAX'],'xkcd:light blue',final['Date'],final['TMIN'],'xkcd:light blue')
plt.gca().fill_between(range(len(final['Date'])), 
                       final['TMIN'], final['TMAX'], 
                       facecolor='xkcd:light blue', 
                       alpha=0.3)

plt.scatter(compare_high['Date'], compare_high['TMAX_y'])
plt.scatter(compare_low['Date'], compare_low['TMIN_y'])

FileNotFoundError: File b'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv' does not exist